In [21]:
import re
import json
from nltk import corpus as corpiss
from transformers import AutoTokenizer


In [4]:
def opener(filename):
    with open(filename) as f:
        contents = f.readlines()
    return contents

In [20]:
f = open('./jeopardy.json')
corpus = json.load(f)
for i,item in enumerate(corpus):
    item['question'] = item['question'][1:-1]
    if re.search('href',item['question']):
        del corpus[i]
        continue
    del item['air_date']
    del item['round']
    del item['show_number']
    if item['value'] is None:
        continue
    item['value'] = int(item['value'][1:].replace(',',''))
corpus[50:52]

[{'category': '"X"s & "O"s',
  'question': 'This 1797 imbroglio began when 3 French agents demanded a huge bribe from U.S. diplomats',
  'value': 2000,
  'answer': 'the XYZ Affair'},
 {'category': 'THE SOLAR SYSTEM',
  'question': 'Objects that pass closer to the sun than Mercury have been named for this mythological figure',
  'value': None,
  'answer': 'Icarus'}]

In [22]:
model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [7]:
# categories = set([i['category']for i in corpus])
# list(categories)[:5]

['MEDICINE & HISTORY',
 'THE LYNDON JOHNSON YEARS',
 'DAWN',
 'CENTRAL PARK ENTERTAINMENT',
 'WHAT A DELICIOUS DISH']

In [34]:
# def tokenize(text,type='question'):
#     if type=='question':
#         text=text[1:-1]
#     text=re.sub('</a>','',re.sub(r'<a href=".*">','',re.sub(r'(\w)([.,?!;:])', r'\1 \2', text))).split()
#     text = [word.lower() for word in text]
#     return text
# # tokenize(corpus[0]['question'])

In [35]:
# questions = [j for j in [tokenize(i['question']) for i in corpus]]
# questions[:2]

[['for',
  'the',
  'last',
  '8',
  'years',
  'of',
  'his',
  'life',
  ',',
  'galileo',
  'was',
  'under',
  'house',
  'arrest',
  'for',
  'espousing',
  'this',
  "man's",
  'theory'],
 ['no',
  '.',
  '2',
  ':',
  '1912',
  'olympian',
  ';',
  'football',
  'star',
  'at',
  'carlisle',
  'indian',
  'school',
  ';',
  '6',
  'mlb',
  'seasons',
  'with',
  'the',
  'reds',
  ',',
  'giants',
  '&',
  'braves']]

In [36]:
# def opener(questions):
#     out = []
#     for idx,question in enumerate(questions):
#         for word in question:
#             out.append(word)
#     return out
# questions_joined = ' '.join(opener(questions))


In [68]:
# def ngrams(text,n):
#     out=[]
#     for i in range(len(text)-n+1):
#         out.append(tuple(text[i:i+n]))
#     return out
# j_qs_bigram=[ngrams(i,2) for i in j_questions]
# j_qs_bigram_flat = []
# for i in j_qs_bigram:
#     for j in i:
#         j_qs_bigram_flat.append(j)

In [80]:
# j_qs_bigram_flat[:25]

[('for', 'the'),
 ('the', 'last'),
 ('last', '8'),
 ('8', 'years'),
 ('years', 'of'),
 ('of', 'his'),
 ('his', 'life'),
 ('life', ','),
 (',', 'galileo'),
 ('galileo', 'was'),
 ('was', 'under'),
 ('under', 'house'),
 ('house', 'arrest'),
 ('arrest', 'for'),
 ('for', 'espousing'),
 ('espousing', 'this'),
 ('this', "man's"),
 ("man's", 'theory'),
 ('no', '.'),
 ('.', '2'),
 ('2', ':'),
 (':', '1912'),
 ('1912', 'olympian'),
 ('olympian', ';'),
 (';', 'football')]

In [110]:
# def ngram_frequency(ngram_list):
#     frequency = {}
#     for ngram in ngram_list:
#         if ngram in frequency:
#             frequency[ngram] += 1
#         else:
#             frequency[ngram] = 1
# #    frequency = sorted(frequency.items(), key=lambda x:x[1], reverse=True)
#     return frequency

In [111]:
# ngrams = ngram_frequency(j_qs_bigram_flat)

In [112]:
# ngrams[:0]
# #x = list(ngrams.values())
# #ngrams.sort()

# list(ngrams)[:3]

[(('of', 'the'), 23153), (('of', 'this'), 19236), (('in', 'the'), 16028)]

In [31]:
# questions_joined[:5]

'for t'

In [13]:
from nltk.lm.preprocessing import pad_both_ends
from nltk import ConditionalFreqDist
from nltk.probability import ConditionalProbDist, ELEProbDist
from nltk.util import pad_sequence
from nltk.lm.preprocessing import pad_both_ends


class NgramModel():

    
    def __init__(self, corpus, n):
        self.n = n
        tokenized_corpus = self._tokenize(corpus)
        self._ngrams = self._build_ngrams(tokenized_corpus, n)
        self._cpd = self._build_distribution(self._ngrams, n)        
        
    def _tokenize(self, corpus):
        
        tokenized_corpus = []
        
        # separate punctuation from previous word
        spaced_corpus = re.sub(r'(\w)([.,?!;:])', r'\1 \2', corpus) 
        
        # split into sentences
        sentences = spaced_corpus.split('.')
        for sentence in sentences:
            words = sentence.split() # split on whitespace
            words = [word.lower() for word in words]
            words = list(pad_both_ends(words, n=self.n))
            tokenized_corpus += words
        
        return tokenized_corpus
            
    def _build_ngrams(self, tokenized_corpus, n):
        n_grams = []
        for i in range(n-1, len(tokenized_corpus)): 
            n_grams.append(tuple(tokenized_corpus[i-(n-1):i+1]))    
        return n_grams
    
    def _build_distribution(self, corpus, n):
               
        cfd = ConditionalFreqDist()
        for ngram in self._ngrams:
            condition = tuple(ngram[0:n-1]) 
            outcome = ngram[n-1]
            
            cfd[condition][outcome] += 1
        bins = len(cfd) # we have to pass the number of bins in our freq dist in as a parameter to probability distribution, so we have a bin for every word
        cpd = ConditionalProbDist(cfd, ELEProbDist, bins)
        self.cpd = cpd
        return cpd
        
    def generate(self, num_sentences = 1, seed = []):
        """
        There are two cases to deal with here. Either we have a start string, or we don't. 
        If we are given a start string, we'll have to find the last n-1 gram and condition on that
        If we are not, we need to generate the first n-1 gram. For a trigram model, we need a bigram. But how can we use our model to generate new words when we have fewer than two words to condition on?
        We can use a bigram model! But wait. If we have a bigram model, how do we generate the first token without another token to condition on? 
        We can use a unigram model! 
        Recursion will save us here. Turns out the easiest way to do this will be to recursively construct an n-1gram model and store it in the main model.
        And how can we 
        Either way, we need a seed condition to enter into the loop with.
        """

        # place to put generated tokens
        string = []

        if seed:
            string = string + (list(pad_sequence(seed, self.n, pad_left=True, pad_right=False, left_pad_symbol='<s>') ) )
        else:
            string = string + (list(pad_sequence('', self.n, pad_left=True, pad_right=False, left_pad_symbol='<s>') ) )
        
        for i in range(num_sentences):
            next_token = tuple(string[-(self.n-1):])
            
            # keep generating tokens as long as we havent reached the stop sequence
            while next_token != '</s>':
                
                # get the last n-1 tokens to condition on next
                lessgram = tuple(string[-(self.n-1):])

    
                next_token = self.cpd[lessgram].generate()
                string.append( next_token )

        string = ' '.join(string)

        return string

        
        

In [37]:
# model = NgramModel(questions_joined,3)


In [38]:
# model.generate(5)

'<s> <s> declare gov </s> </s> <s> <s> ludwig roselius\' research into the heart with oxygen in 1988 is tv\'s twinkie spokesman 1 of it term for reformers like wm </s> </s> <s> <s> female speed-skating gold medalist traded in one leap give up its sos & radioed back , situation room &, of course has his own star on her brother\'s marriage this "firebird" composer\'s opera "rigoletto" was titled "ain\'t misbehavin\'" in the college championship contestant margaret bickers , a deceived person is from webster groves , hills and fields , baby doc duvalier succeeded papa doc duvalier fled haiti a cylindrical package , & we could save the jews ; hey are known officially as the actor playing petruchio in "the seduction of joe clark\'s unusual tools in 2004 scarlett hit the longest ever of the sheet </s>'

In [44]:
# from transformers import pipeline

# model_checkpoint = "consciousAI/question-answering-roberta-base-s-v2"

# # context = """
# # 🤗 Transformers is backed by the three most popular deep learning libraries — Jax, PyTorch and TensorFlow — with a seamless integration
# # between them. It's straightforward to train your models with one before loading them for inference with the other.
# # """
# # question = "Which deep learning libraries back 🤗 Transformers?"

# # question_answerer = pipeline("question-answering", model=model_checkpoint)
# # question_answerer(question=question, context=context)
# question_answerer = pipeline("question-answering",model=model_checkpoint)
# question(answerer